<a href="https://colab.research.google.com/github/mashuai191/pantograph/blob/master/Pantograph_Demo_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# how to 
# how to run web app in google colab
    #refer to : https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b


In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 723.9MB 25kB/s 
     |████████████████████████████████| 4.1MB 89.1MB/s 
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0
     |████████████████████████████████| 276kB 37.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=fc002ac8e92318141d0eb6397e1ca8ade8b3fb36f849094c4e4851178a402630
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-kcx9zsgj
  Running command git

In [0]:
!rm -rf detectron2_repo
!git clone https://github.com/mashuai191/detectron2 detectron2_repo

!pip install -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 3563, done.
remote: Total 3563 (delta 0), reused 0 (delta 0), pack-reused 3563
Receiving objects: 100% (3563/3563), 2.14 MiB | 3.79 MiB/s, done.
Resolving deltas: 100% (2458/2458), done.
Obtaining file:///content/detectron2_repo
  Created wheel for fvcore: filename=fvcore-0.1.dev200325-cp36-none-any.whl size=38930 sha256=a9d1d69047b8d3d5438772c17354ff39977675857ee9902e61df208b9f892656
  Stored in directory: /root/.cache/pip/wheels/58/4f/d1/6b0ada294145df5fe73092d9849d27be5b5b720fd8cfdd1b98
Successfully built fvcore
  Running setup.py develop for detectron2


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [0]:
!pip install flask-ngrok


In [0]:
import atexit
import json
import os
import platform
import shutil
import subprocess
import tempfile
import time
import zipfile
from pathlib import Path
from threading import Timer

import requests


def _get_command():
    system = platform.system()
    if system == "Darwin":
        command = "ngrok"
    elif system == "Windows":
        command = "ngrok.exe"
    elif system == "Linux":
        command = "ngrok"
    else:
        raise Exception("{system} is not supported".format(system=system))
    return command


def _run_ngrok(port):
    command = _get_command()
    ngrok_path = str(Path(tempfile.gettempdir(), "ngrok"))
    _download_ngrok(ngrok_path)
    executable = str(Path(ngrok_path, command))
    os.chmod(executable, 0o777)
    ngrok = subprocess.Popen([executable, 'http', str(port)])
    atexit.register(ngrok.terminate)
    localhost_url = "http://localhost:4040/api/tunnels"  # Url with tunnel details
    time.sleep(1)
    tunnel_url = requests.get(localhost_url).text  # Get the tunnel information
    j = json.loads(tunnel_url)

    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
    tunnel_url = tunnel_url.replace("https", "http")
    return tunnel_url


def _download_ngrok(ngrok_path):
    if Path(ngrok_path).exists():
        return
    system = platform.system()
    if system == "Darwin":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-darwin-amd64.zip"
    elif system == "Windows":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-amd64.zip"
    elif system == "Linux":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"
    else:
        raise Exception(f"{system} is not supported")
    download_path = _download_file(url)
    with zipfile.ZipFile(download_path, "r") as zip_ref:
        zip_ref.extractall(ngrok_path)


def _download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    download_path = str(Path(tempfile.gettempdir(), local_filename))
    with open(download_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
    return download_path


def start_ngrok(port):
    ngrok_address = _run_ngrok(port)
    print(f" * Running on {ngrok_address}")
    print(f" * Traffic stats available on http://127.0.0.1:4040")


def run_with_ngrok(app):
    """
    The provided Flask app will be securely exposed to the public internet via ngrok when run,
    and the its ngrok address will be printed to stdout
    :param app: a Flask application object
    :return: None
    """
    old_run = app.run

    def new_run(*args, **kwargs):
        port = kwargs.get('port', 5000)
        thread = Timer(1, start_ngrok, args=(port,))
        thread.setDaemon(True)
        thread.start()
        old_run(*args, **kwargs)
    app.run = new_run

In [0]:
#from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"
  
#app.run()

In [0]:

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import cv2
import os

DIR_BASE = '/content/gdrive/My Drive/App/machine_learning/cv/shanghaidianqi_line5'
DIR_DATASET = '20181117_03_08_1750_G1380.MP4'


In [0]:
!git clone https://github.com/mashuai191/flask-video-streaming.git

Cloning into 'flask-video-streaming'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 95 (delta 4), reused 4 (delta 1), pack-reused 79
Unpacking objects: 100% (95/95), done.


In [0]:
%cd flask-video-streaming

/content/flask-video-streaming


In [0]:
!ls


1.jpg	base_camera.py	  LICENSE		   requirements.txt
2.jpg	camera_opencv.py  README.md		   templates
3.jpg	camera_pi.py	  requirements-opencv.txt
app.py	camera.py	  requirements-pi.txt


In [0]:
import os
import cv2
from base_camera import BaseCamera


class Camera(BaseCamera):
    video_source = 0

    def __init__(self):
        if os.environ.get('OPENCV_CAMERA_SOURCE'):
            Camera.set_video_source(int(os.environ['OPENCV_CAMERA_SOURCE']))
        super(Camera, self).__init__()

    @staticmethod
    def set_video_source(source):
        Camera.video_source = source

    @staticmethod
    def frames():
        #camera = cv2.VideoCapture(Camera.video_source)
        camera = cv2.VideoCapture(os.path.join(DIR_BASE, DIR_DATASET))
        if not camera.isOpened():
            raise RuntimeError('Could not start camera.')

        MetadataCatalog.get("balloon_" + 'test').set(thing_classes=["balloon"])
        cfg = get_cfg()
        # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
        # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
        #cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        #cfg.OUTPUT_DIR = "pre-trained models/model_final_R50_x3_ratio012.pth"
        cfg.MODEL.WEIGHTS = os.path.join(DIR_BASE, "trained_models/model_final_R50_x3_ratio012.pth")
        cfg.DATASETS.TEST = ("balloon_test", )
        cfg.DATASETS.TRAIN = ("balloon_test", )
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

        predictor = DefaultPredictor(cfg)

        while True:
            # read current frame
            _, img = camera.read()
            cropped_image = img[200:625, 200:1150]
            print (img.shape, cropped_image.shape)
            

            # let's predict

            outputs = predictor(cropped_image)

            v = Visualizer(cropped_image[:, :, ::-1], metadata=MetadataCatalog.get("balloon_" + 'test'), scale=1.2)
            v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            img_tmp = v.get_image()[:, :, ::-1]
            #cv2_imshow(v.get_image()[:, :, ::-1])


            imS = cv2.resize(img_tmp, (350, 250))
            #print (imS.shape)

            # encode as a jpeg image and return it
            yield cv2.imencode('.jpg', imS)[1].tobytes()


In [0]:
#!/usr/bin/env python
from importlib import import_module
import os
from flask import Flask, render_template, Response

# import camera driver
if os.environ.get('CAMERA'):
    Camera = import_module('camera_' + os.environ['CAMERA']).Camera
#else:
#    from camera import Camera

# Raspberry Pi camera module (requires picamera package)
# from camera_pi import Camera

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run


@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')


def gen(camera):
    """Video streaming generator function."""
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    print ('video_feed pid {}', os.getpid())
    return Response(gen(Camera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/video_feed1')
def video_feed1():
    """Video streaming route. Put this in the src attribute of an img tag."""
    print ('video_feed1, pid {}', os.getpid())
    return Response(gen(Camera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

#if __name__ == '__main__':
#    app.run(host='0.0.0.0', threaded=True)
#from flask.flask_script import Manager

#manager = Manager(app)

#if __name__ == "__main__":
#    manager.run(host='0.0.0.0', threaded=True)
app.run(host='0.0.0.0', threaded=False, processes=2)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


 * Running on http://68db5d08.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Mar/2020 14:53:53] "GET / HTTP/1.1" 200 -


video_feed pid {} 2675
Camera init.
Starting camera thread.
pid  2675 tid  139886168557440
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)


127.0.0.1 - - [25/Mar/2020 14:53:59] "GET /video_feed HTTP/1.1" 200 -


(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)


127.0.0.1 - - [25/Mar/2020 14:54:00] "GET /favicon.ico HTTP/1.1" 404 -


(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 1920, 3) (425, 950, 3)
(1080, 192